# Estudio de los datos de Ventas de una Ferretería

Autor: Diana Chacón Ocariz

## Contexto:

Se trata de una pequeña ferretería que maneja un poco más de 3.000 productos distintos. Poseen un software de gestión genérico que les provee una gran cantidad de reportes, básicamente tablas con números, díficiles de analizar (un reporte puede constar de varias decenas de páginas).


## Objetivos del negocio:

**Tener mayor visibilidad sobre las ventas para poder mejorar el proceso de compras y la toma de decisiones en general:** 

    - Poder analizar objetivamente las ventas
    - Determinar los productos que podrían entrar en rotura de stock al final del período
    - Identificar los productos menos vendidos
    - Identificar patrones en el comportamiento de las ventas para poder hacer predicciones de ventas
    

## Objetivos académicos:

    - Estudiar un caso real, con datos reales y cuyo resultado pueda ayudar a alguien a resolver un problema 
    - Demostrar que la ciencia de datos también puede ayudar a las PYMES
    - Conocer y practicar el uso de herramientas de ciencia de datos
    
## Fuentes de datos:

Los datos provienen de reportes sacados del software de gestión de la empresa. Se trata de archivos .xls que contienen sólo los datos de reportes sobre ventas por producto (2021 y 2022) y el stock al final del período. 

# Notebook 2: EDA: Análisis y Visualización

Recuperamos los datos (ya limpios) de los archivos **parquet** para comenzar un análisis más profundo de los datos, responder preguntas del negocio, verificar si hay patrones en las ventas y preparar los datos para utilizarlos en modelos predictivos.

Utilizaremos varias librerías de visualización con el fin de compararlas: **Matplotlib, Seaborn y Altair**

In [1]:
# Librerías utilizadas

import os
import glob
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

import altair as alt

In [2]:
BASE_DIR = Path.cwd()
BASE_DIR

PosixPath('/home/diana/Documentos/Ciencia de Datos/Proyecto Ventas')

In [3]:
%%time 
df_ventas = pd.read_parquet(f"{BASE_DIR / 'datos/out/ventas.parquet'}", engine='fastparquet')
df_ventas

CPU times: user 2.69 s, sys: 46.3 ms, total: 2.74 s
Wall time: 3.17 s


,num,fecha_comp,cliente,vendedor,cod,producto,cantidad,monto,tipo,fecha,tasa_dolar,monto_dolar
0,0000006016,2022-01-03 08:38:00,29674026.0,14,00001,PEGA DE CONTACTO/PEGA ZAPATERA (90 ML) ENVASADO,1.0,8.64,ne,2022-01-03,4.800000,1.800000
1,0000006035,2022-01-03 10:55:00,13763886.0,7,00005,"NIPLE PLASTICO 1"" * 13CMS",2.0,11.52,ne,2022-01-03,4.800000,2.400000
2,0000006020,2022-01-03 09:24:00,14281493.0,7,00008,ALAMBRE C-17.5 LISO HG KILOGRAMO VICSON,20.0,358.60,ne,2022-01-03,4.800000,74.708333
3,0000006052,2022-01-03 14:39:00,19339734.0,7,00008,ALAMBRE C-17.5 LISO HG KILOGRAMO VICSON,1.0,17.93,ne,2022-01-03,4.800000,3.735417
4,0000006060,2022-01-03 15:23:00,15760108.0,7,00028,TEFLON PROF 1/2 * 10mts SUPER EXTRA LUZ/P.T.F.E,1.0,1.92,ne,2022-01-03,4.800000,0.400000
...,...,...,...,...,...,...,...,...,...,...,...,...
30684,0000002772,2021-09-18 11:41:00,9330053.0,10,05279,GRIFERIA LAVAMANOS PLASTICA CROMADA POMO CAMP...,1.0,14.35,ne,2021-09-18,3.962347,3.621591
30685,0000002763,2021-09-18 10:30:00,9127920.0,7,05297,FOTOCELDA 120V FCE-124 VERT,1.0,26.65,ne,2021-09-18,3.962347,6.725811
30686,0000002763,2021-09-18 10:30:00,9127920.0,7,05298,BASE PARA FOTOCELDA ESTANDAR RECEPTACULO,1.0,16.40,ne,2021-09-18,3.962347,4.138961
30687,0000002765,2021-09-18 10:36:00,80457156.0,13,05345,CONFITERIA MINICHIPS FESTIVAL,1.0,1.63,ne,2021-09-18,3.962347,0.411372


# Análisis de las Ventas

# Variación de las Ventas en el tiempo

In [4]:
df_ventas_fecha = df_ventas.loc[:,['fecha', 'num', 'cantidad', 'monto_dolar']]
df_ventas_fecha.set_index('fecha', inplace=True)
df_ventas_fecha

,num,cantidad,monto_dolar
fecha,,,
2022-01-03,0000006016,1.0,1.800000
2022-01-03,0000006035,2.0,2.400000
2022-01-03,0000006020,20.0,74.708333
2022-01-03,0000006052,1.0,3.735417
2022-01-03,0000006060,1.0,0.400000
...,...,...,...
2021-09-18,0000002772,1.0,3.621591
2021-09-18,0000002763,1.0,6.725811
2021-09-18,0000002763,1.0,4.138961


### Conclusiones:

    1) En promedio de venden 409 productos cada día
    2) El promedio de ventas diario es de 673 dólares
    3) 41 es el promedio de facturas que se hacen diariamente
    4) Se nota un incremento de las ventas a partir del 2do semestre del 2021
    5) Entre marzo y junio de 2021 se resgistraron la menor cantidad de ventas del período estudiado

## Evolución de las Ventas Mes a Mes

In [5]:
# Utilizamos nunique para contabilizar sólo una vez cada referencia de factura
df_ventas_ag = df_ventas.pivot_table(index='fecha', values=['num', 'cantidad', 'monto_dolar'], 
                                        aggfunc={'num':'nunique', 'cantidad':sum, 'monto_dolar':sum })
df_ventas_ag.reset_index(inplace=True)
df_ventas_ag

,fecha,cantidad,monto_dolar,num
0,2021-01-05,609.40,926.821704,49
1,2021-01-06,335.98,292.536671,48
2,2021-01-07,305.85,496.648389,48
3,2021-01-08,264.00,377.470866,42
4,2021-01-09,154.00,98.059123,17
...,...,...,...,...
329,2022-02-10,723.35,1881.088172,51
330,2022-02-11,294.39,771.884120,48
331,2022-02-12,228.90,281.356223,37
332,2022-02-14,511.30,1804.905172,65


In [6]:
df_ventas_ag['mes_anio'] = df_ventas_ag.fecha.dt.strftime('%m-%Y')
df_ventas_ag['dia_semana'] = df_ventas_ag.fecha.dt.weekday
df_ventas_ag['dia_mes'] = df_ventas_ag.fecha.dt.day
df_ventas_ag

,fecha,cantidad,monto_dolar,num,mes_anio,dia_semana,dia_mes
0,2021-01-05,609.40,926.821704,49,01-2021,1,5
1,2021-01-06,335.98,292.536671,48,01-2021,2,6
2,2021-01-07,305.85,496.648389,48,01-2021,3,7
3,2021-01-08,264.00,377.470866,42,01-2021,4,8
4,2021-01-09,154.00,98.059123,17,01-2021,5,9
...,...,...,...,...,...,...,...
329,2022-02-10,723.35,1881.088172,51,02-2022,3,10
330,2022-02-11,294.39,771.884120,48,02-2022,4,11
331,2022-02-12,228.90,281.356223,37,02-2022,5,12
332,2022-02-14,511.30,1804.905172,65,02-2022,0,14


In [7]:
lineas = alt.Chart(df_ventas_ag).mark_line().encode(
    x='fecha:T',
    y='monto_dolar:Q',
    color=alt.Color('yearmonth(fecha):O', scale=alt.Scale(scheme='goldgreen')),
    tooltip=[
        alt.Tooltip('fecha:T', title='Fecha'),
        alt.Tooltip('monto_dolar:Q', title='Ventas en $')
    ]).properties(width=800, height=200)

lineas

alt.Chart(...)

In [8]:
lineas = alt.Chart(df_ventas_ag).mark_line().encode(
    x='fecha:T',
    y='cantidad:Q',
    color=alt.Color('yearmonth(fecha):O', scale=alt.Scale(scheme='purpleblue')),
    tooltip=[
        alt.Tooltip('fecha:T', title='Fecha'),
        alt.Tooltip('cantidad:Q', title='Volumen de Venta')
    ]).properties(width=800, height=200)

lineas

alt.Chart(...)

In [9]:
lineas = alt.Chart(df_ventas_ag).mark_line().encode(
    x='fecha:T',
    y='num:Q',
    color=alt.Color('yearmonth(fecha):O', scale=alt.Scale(scheme='goldorange')),
    tooltip=[
        alt.Tooltip('fecha:T', title='Fecha'),
        alt.Tooltip('num:Q', title='N° Facturas')
    ]).properties(width=800, height=200)

lineas

alt.Chart(...)

In [10]:
df_ventas_dia_semana = df_ventas_ag.pivot_table(index='mes_anio',
                                            columns='dia_semana', aggfunc={'monto_dolar': sum},
                                            fill_value=0)
df_ventas_dia_semana 

monto_dolar                                                      \
dia_semana            0            1            2            3            4   
mes_anio                                                                      
01-2021     2784.790848  3724.941735  1904.395292  2664.156668  2691.912089   
01-2022     4607.765117  4840.628789  3014.172976  2680.853759  3390.207444   
02-2021     1487.808610  3409.444342  3009.591629  3321.042178  3099.457197   
02-2022     2851.047414  3621.524281  2150.804754  2705.217959  3155.767593   
03-2021     3154.429796  2621.734764  2409.412415  2697.310711  1542.804509   
04-2021     2347.962698  3028.319106  2202.769910  1937.128764  2816.807784   
05-2021     2726.828498  1552.428012  5490.038652  1368.441352  1598.738326   
06-2021     3409.068815  3349.947459  2794.266252  1427.249511  2294.084378   
07-2021     2742.526508  3622.278674  2897.440111  3385.056171  4102.983283   
08-2021     3813.911816  3634.343572  3457.868795  1777.697439  1811.015563   
09-2021     3122.104340  4045.390292  3559.772908  3265.433007  2535.006467   
10-2021     5281.097548  3027.236408  4577.617768  2670.035010  2706.640421   
11-2021     4282.041414  4223.213901  3041.794665  2553.439244  3361.564040   
12-2021     3455.586364  2677.405213  4393.111531  3152.304079  4916.284466   

                         
dia_semana            5  
mes_anio                 
01-2021      943.314263  
01-2022     3926.310072  
02-2021     1108.215054  
02-2022      798.153199  
03-2021      467.503788  
04-2021      421.100324  
05-2021      777.268368  
06-2021      870.838121  
07-2021     1760.444940  
08-2021     1029.554831  
09-2021     1074.564331  
10-2021     3073.494269  
11-2021     1528.143621  
12-2021      886.941365

In [11]:
mapa = alt.Chart(df_ventas_ag).mark_rect().encode(
                x=alt.X('day(fecha):T', title='Días de la Semana', axis = alt.Axis(labelAngle=0, labelFontSize=14)),
                y=alt.Y('yearmonth(fecha):T', title = 'Ventas en $', scale=alt.Scale(zero=False), 
                  axis = alt.Axis(grid=True, titleAnchor='middle', titleAngle = 270, labelFontSize=10)),
                color=alt.Color('sum(monto_dolar):Q', scale=alt.Scale(scheme='goldgreen'), title='Ventas en $'),
                tooltip=[
                    alt.Tooltip('day(fecha):T', title='Día'),
                    alt.Tooltip('yearmonth(fecha):T', title='Mes y Año'),
                    alt.Tooltip('sum(monto_dolar):Q', title='Ventas en $')]
                ).properties(title='Ventas en $ por Día de la Semana',
                             width=300, 
                             height=300
                ).configure_title(
                    fontSize = 16,
                    anchor = 'middle'
                ).interactive()

mapa

alt.Chart(...)

In [12]:
barras = alt.Chart(df_ventas_ag).mark_bar().encode(
                x=alt.X('day(fecha):T', title='Días de la Semana', axis = alt.Axis(labelAngle=0, labelFontSize=14)),
                y=alt.Y('sum(monto_dolar):Q', title = 'Ventas en $', scale=alt.Scale(zero=False), 
                  axis = alt.Axis(grid=True, titleAnchor='middle', titleAngle = 270, labelFontSize=10)),
                color=alt.Color(
                    'sum(monto_dolar):Q', scale=alt.Scale(scheme='goldgreen'), title='Ventas en $'),
                tooltip=[
                    alt.Tooltip('day(fecha):T', title='Día'),
                    alt.Tooltip('sum(monto_dolar):Q', title='Ventas en $')],
                ).properties(title='Ventas en $ por Día de la Semana',
                             width=600, 
                             height=300
                ).configure_title(
                    fontSize = 16,
                    anchor = 'middle'
                ).interactive()
barras

alt.Chart(...)

In [13]:
df_ventas_dia_mes = df_ventas_ag.pivot_table(index='mes_anio',
                                            columns='dia_mes', aggfunc={'monto_dolar': sum},
                                            fill_value=0)
df_ventas_dia_mes

monto_dolar                                                      \
dia_mes            1            2            3            4            5    
mes_anio                                                                    
01-2021      0.000000     0.000000     0.000000     0.000000   926.821704   
01-2022      0.000000     0.000000  1591.593750   758.901031   837.132114   
02-2021    576.868495   877.654503   426.736569   651.904401  1398.064662   
02-2022   1756.324895   755.778723   824.129787  2383.883473   516.796976   
03-2021   1247.560550   310.635069   502.960760  1207.091429   590.472359   
04-2021      0.000000     0.000000     0.000000     0.000000   268.741041   
05-2021      0.000000     0.000000   169.544003   220.616036   291.866310   
06-2021   1563.823121   612.552178   591.345295   637.657822   168.589568   
07-2021    416.055473   650.871448   197.589427     0.000000   634.166975   
08-2021      0.000000  2028.349967   504.849055   873.107250     0.000000   
09-2021    338.374591   391.866812   294.479298   274.037580     0.000000   
10-2021    579.193280  1742.244297     0.000000  2291.216630   853.060533   
11-2021    500.472527   785.301339   619.440529   907.849890   440.254425   
12-2021   1754.004032   433.751553  1135.519833   259.908524     0.000000   

                                                                           \
dia_mes            6            7            8            9            10   
mes_anio                                                                    
01-2021    292.536671   496.648389   377.470866    98.059123     0.000000   
01-2022   1569.805274   987.729675   694.787286     0.000000  1230.453988   
02-2021    274.759691     0.000000   507.689803   995.648242   914.567090   
02-2022      0.000000  1046.142241   875.757576  1395.026030  1881.088172   
03-2021    134.006455     0.000000   518.569803   668.978256   532.901499   
04-2021    559.966597   271.320995   419.808247   235.351498   139.390734   
05-2021    224.324624   119.475605   135.584869     0.000000   324.692543   
06-2021      0.000000   659.694432   443.874941   389.142882   273.577969   
07-2021   1338.564721  1224.456318  1348.962770   937.139976   419.200390   
08-2021      0.000000   508.017601     0.000000   683.950377   544.170909   
09-2021    470.278577  1921.822259  1448.564735   623.570036   750.432271   
10-2021   1351.338235   623.452785   780.975787   417.384236     0.000000   
11-2021    320.924444     0.000000   523.150110   225.858388   848.488017   
12-2021   1062.589958  1076.601677   757.338205  1231.157563  1387.858650   

          ...                                                                  \
dia_mes   ...          22           23           24           25           26   
mes_anio  ...                                                                   
01-2021   ...  663.691129   230.979657     0.000000  1020.231259  1422.078627   
01-2022   ...  355.457557     0.000000   660.230290    66.785863     7.997908   
02-2021   ...  403.250313  1536.141598  1089.783277   601.951645   698.116953   
02-2022   ...    0.000000     0.000000     0.000000     0.000000     0.000000   
03-2021   ...  280.979318   637.937824   299.366883   673.760794   407.773434   
04-2021   ...  508.427792   926.682998   164.130460     0.000000  1236.686858   
05-2021   ...  145.019230     0.000000   562.645354   299.939176   455.189314   
06-2021   ...  405.628004   854.401915   208.702323   471.923384   205.354236   
07-2021   ...  488.747125   376.931406   199.702662     0.000000   727.887505   
08-2021   ...    0.000000   347.174693  1815.168917   283.621166   849.900336   
09-2021   ...  673.555403  1151.237664   881.810325   359.972220     0.000000   
10-2021   ...  599.594966   495.002299     0.000000  1800.578341   421.847191   
11-2021   ...  670.511066   448.669405   562.609218   438.017893   563.803922   
12-2021   ...  516.195699   198.163441     0.000000     0.000000     0.000000   

            

In [14]:
mapa = alt.Chart(df_ventas_ag).mark_rect().encode(
                x=alt.X('dia_mes:O', title='Días del mes', axis = alt.Axis(labelAngle=0, labelFontSize=14)),
                y=alt.Y('yearmonth(fecha):T', title = 'Mes y Año', scale=alt.Scale(zero=False), 
                  axis = alt.Axis(grid=True, titleAnchor='middle', titleAngle = 270, labelFontSize=10)),
                color=alt.Color(
                    'sum(monto_dolar):Q', scale=alt.Scale(scheme='goldgreen'), title='Ventas en $'),
                tooltip=[
                    alt.Tooltip('dia_mes:O', title='Día'),
                    alt.Tooltip('yearmonth(fecha):T', title='Mes y Año'),
                    alt.Tooltip('sum(monto_dolar):Q', format=',.2f', title='Ventas en $'),
                    alt.Tooltip('cantidad:Q', format=',.2f', title='Volumen de Ventas'),
                    alt.Tooltip('num:Q', title='N° Facturas')]
                ).properties(title='Ventas en $ por Día del Mes',
                             width=800,  
                             height=300
                ).configure_title(
                    fontSize = 16,
                    anchor = 'middle'
                ).interactive()

mapa

alt.Chart(...)

In [15]:
barras = alt.Chart(df_ventas_ag).mark_bar().encode(
                x=alt.X('dia_mes:O', title='Días del Mes', axis = alt.Axis(labelAngle=0, labelFontSize=14)),
                y=alt.Y('sum(monto_dolar):Q', title = 'Ventas en $', scale=alt.Scale(zero=False), 
                  axis = alt.Axis(grid=True, titleAnchor='middle', titleAngle = 270, labelFontSize=10)),
                color=alt.Color(
                    'sum(monto_dolar):Q', scale=alt.Scale(scheme='goldgreen'), title='Ventas en $'),
                tooltip=[
                    alt.Tooltip('dia_mes:O', title='Día'),
                    alt.Tooltip('sum(monto_dolar):Q', title='Ventas en $')],
                ).properties(title='Ventas en $ por Día del Mes',
                             width=800, 
                             height=300
                ).configure_title(
                    fontSize = 16,
                    anchor = 'middle'
                ).interactive()
barras

alt.Chart(...)

Los lunes y martes son los días en los que más se vende en $. El viernes en Volumen

Los jueves son los días que menos se vende, que sea en volumen o en $

### Análisis por mes

In [16]:
lineas = alt.Chart(df_ventas_ag).mark_bar().encode(
    x='dia_mes:O',
    y='monto_dolar:Q',
    color=alt.Color('yearmonth(fecha):T', scale=alt.Scale(scheme='goldgreen')),
    column=alt.Column('yearmonth(fecha):T', title='yearmonth(fecha)'),
    tooltip=[
        alt.Tooltip('fecha:T', title='Fecha'),
        alt.Tooltip('monto_dolar:Q', title='Ventas en $'),
    ]).properties(width=800, height=200)

lineas

alt.Chart(...)